In [48]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import dateutil
import analysis
import itertools

packets = analysis.parse_packet_file('data/all.txt')
packets += analysis.parse_packet_file('data/temp_basals.txt')


# Select valid packets of a particular length
packets = filter(lambda x: x.is_valid() and x.body_len == 10, packets)

# Uncomment if you want to work with less data (faster while developing)
packets = packets[0:100]

In [49]:
def diff(m1,m2):
    xored_bytes = [ord(a) ^ ord(b) for a,b in zip(m1,m2)]
    return xored_bytes

def bits(data):
    return np.unpackbits(np.frombuffer(data, dtype=np.uint8))

def join_ints(ints):
    return ':'.join(str(x) for x in ints)

In [61]:
class DiffMessage(object):

    # data is passed in like this: ((m1,c1),(m2,c2))
    def __init__(self, combination):
        self.m1 = combination[0][0]
        self.c1 = combination[0][1]
        self.m2 = combination[1][0]
        self.c2 = combination[1][1]
        
        # diffs
        self.dm = diff(self.m1, self.m2)
        self.dc = diff(self.c1, self.c2)
        
        # diff stats
        diff_bits = np.unpackbits(np.array(self.dm, dtype=np.uint8))
        self.diff_bits_count = diff_bits.sum()
        self.diff_bits_key = join_ints(np.where(diff_bits)[0].tolist())
        
        self.confirmations = 0
        self.conflicts = 0
        
    def update_observation(self, other):
        if other.dc == self.dc:
            self.confirmations += 1
        else:
            self.conflicts += 1
            
    def __str__(self):
        return "%s: msg=%s, crc=%s" % (self.diff_bits_key, self.dm, self.dc)
    


In [4]:
print packets[0]

2016-06-26T20:33:28.412197 ID1:1f01482a PTYPE:PDM SEQ:13 ID2:1f01482a B9:10 BLEN:3 MTYPE:0e01 BODY:00802c CRC:88


In [5]:
print packets[0].tx_data().encode('hex')

1f01482aad1f01482a10030e0100802c88


In [67]:
# This expresses our theory about which bits are CRC, and which bits
# are included in CRC calculation.  Takes a full message and splits it
# into message,crc
def parts(data):
    msg = data[5:-3] # Everything except ID1, byte 4 (sequence & flags), packet crc, and 16bit chksum
    chksum = data[-3:-1]
    return (msg, chksum)

data = [parts(p.tx_data()) for p in packets]
#diffs = [DiffMessage(c) for c in itertools.combinations(data, 2)]

cracked_bits_dict = {}
for c in itertools.combinations(data, 2):
    d = DiffMessage(c)
    if d.diff_bits_count == 0:
        continue
    if d.diff_bits_key in cracked_bits_dict:
        cracked_bits_dict[d.diff_bits_key].update_observation(d)
    else:
        cracked_bits_dict[d.diff_bits_key] = d
        


In [68]:
[key for key in cracked_bits_dict.keys() if len(key.split(':')) == 1]

['36', '35', '34']

In [76]:
print cracked_bits_dict['36:80:115']

36:80:115: msg=[0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 128, 0, 0, 0, 16, 0], crc=[129, 61]


In [73]:
[key for key in cracked_bits_dict.keys() if len(key.split(':')) == 2]

['35:36',
 '35:80',
 '78:114',
 '80:116',
 '80:115',
 '34:35',
 '34:36',
 '35:37',
 '34:117',
 '34:116',
 '36:116',
 '35:116',
 '35:117',
 '36:37']

In [56]:
[key for key in cracked_bits_dict.keys() if len(key.split(':')) == 3]

['36:78:114',
 '34:37:117',
 '36:80:116',
 '36:80:115',
 '79:115:117',
 '79:115:116',
 '35:36:37',
 '35:115:116',
 '79:80:114',
 '34:36:117',
 '34:36:116',
 '34:35:80',
 '36:37:117',
 '35:37:117',
 '35:36:80',
 '34:35:117',
 '36:79:80',
 '34:116:117',
 '79:114:116',
 '79:114:117',
 '79:114:115',
 '34:35:36',
 '34:36:80',
 '34:80:115',
 '34:80:116',
 '34:80:117',
 '35:36:116',
 '35:79:115',
 '36:116:117',
 '35:116:117',
 '36:79:115',
 '35:80:116',
 '35:36:117',
 '80:115:117']